In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.xarray
import hvplot.pandas

import geopandas as gpd

import cartopy.crs as ccrs
import geoviews.feature as gf
import geoviews as gv

import xopr
import xopr.bedmap

In [ ]:
hv.extension('matplotlib')

epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
coastline = gf.coastline.options(scale='50m').opts(projection=epsg_3031)

epsg_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)
coastline_greenland = gf.coastline.options(scale='50m').opts(projection=epsg_3413)

xopr.bedmap.fetch_bedmap()

In [ ]:
opr = xopr.OPRConnection(cache_dir='radar_cache')

In [ ]:
antarctica = xopr.geometry.get_antarctic_regions(merge_regions=True)
opr_antarctica_df = opr.query_frames(geometry=antarctica)
opr_antarctica_df = opr_antarctica_df.to_crs('EPSG:3031')

In [ ]:
# Calculate total length of frames_antarctica
opr_antarctica_df.length.sum() / 1000

length_by_collection = opr_antarctica_df.assign(length_km=opr_antarctica_df.length/1000).groupby('collection')['length_km'].sum()
length_by_collection[length_by_collection.index.str.contains(r'Basler[JM]KB')].sum()

In [ ]:
df = xopr.bedmap.query_bedmap_catalog()

utig_seasons_upcoming = [
    'UTIG_2008_ICECAP_AIR_BM2',
    'UTIG_2015_EAGLE_AIR_BM3'
]

df_utig = df[df['id'].isin(utig_seasons_upcoming)]

# Find all rows of df where the 'id' column contains 'AWI' 
bedmap_awi = df[df['id'].str.contains('AWI')]

# Find all rows where the 'id' column contains 'LDEO'
bedmap_ldeo = df[df['id'].str.contains('LDEO_2015_ROSETTA')]

inprogress_opts = {
    'linewidth': 0.5,
    'color': 'tab:orange',
    'alpha': 0.2
}

p = coastline
p *= bedmap_awi.to_crs('EPSG:3031').hvplot(**inprogress_opts)
p *= bedmap_ldeo.to_crs('EPSG:3031').hvplot(**inprogress_opts)
p *= df_utig.to_crs('EPSG:3031').hvplot(**inprogress_opts).relabel('Coming Soon')
p *= opr_antarctica_df.hvplot(color='tab:blue', linewidth=0.5).relabel('Available today')
p = p.opts(fig_size=500, fontsize=14)
# Make background transparent
p = p.opts(bgcolor=(1,1,1,0), fig_alpha=0)
p

In [ ]:
hvplot.save(p, 'outputs/agu25/opr_data_map_antarctica.png', dpi=500)

In [ ]:
greenland = xopr.geometry.get_greenland_regions(merge_regions=True)
opr_greenland_df = opr.query_frames(geometry=greenland)
opr_greenland_df = opr_greenland_df.to_crs('EPSG:3413')

In [ ]:
p_greenland = coastline_greenland
p_greenland *= opr_greenland_df.hvplot(color='tab:blue', linewidth=0.5)
p_greenland = p_greenland.opts(fig_size=200, fontsize=14)
p_greenland = p_greenland.opts(bgcolor=(1,1,1,0), fig_alpha=0)
p_greenland

In [ ]:
hvplot.save(p_greenland, 'outputs/agu25/opr_data_map_greenland.png', dpi=500)

## Simple demo

In [ ]:
# Setup an OPR connection object and, optionally, a local cache directory
opr = xopr.OPRConnection(cache_dir='radar_cache')

# Select a region of interest and create a plot of the region
thwaites = xopr.geometry.get_antarctic_regions(name='Thwaites')
thwaites_outline = gpd.GeoDataFrame(geometry=[thwaites], crs='EPSG:4326').to_crs('EPSG:3031')
p = thwaites_outline.hvplot(linewidth=2, facecolor='#98e7eaff', alpha=0.5)

# Query OPR for frames that intersect the Thwaites region and add them to the plot
opr_thwaites_df = opr.query_frames(geometry=thwaites).to_crs('EPSG:3031')
p *= opr_thwaites_df.hvplot(color='#032f56', linewidth=0.5)

In [ ]:
p = p.opts(xlim=(-1.7e6, -0.8e6), xaxis=None, yaxis=None, aspect='equal')
hvplot.save(p, 'outputs/agu25/thwaites_map.png', dpi=300)
p

In [ ]:
# Filter for items where the index matches 20111112_02


#df.to_crs('EPSG:3031').hvplot()


In [ ]:
df = opr.query_frames(collections=['2011_Antarctica_DC8'], segment_paths='20111112_02')[17:23]
segment_ds = opr.load_frames(df, merge_flights=True, data_product='CSARP_standard')
segment_ds

In [ ]:
stacked = segment_ds.resample(slow_time='2s').mean()
stacked = xopr.interpolate_to_vertical_grid(stacked, vertical_coordinate='wgs84')
stacked = xopr.radar_util.add_along_track(stacked)
stacked = stacked['Data'].resample(slow_time='2s').mean()
stacked = 10*np.log10(np.abs(stacked))
stacked.hvplot.image(cmap='gray', clim=(-170, None), ylim=(-2500, None))

In [ ]:
p = stacked.hvplot.image(cmap='gray', clim=(-170, None), ylim=(-2500, None))
hvplot.save(p, 'outputs/agu25/thwaites_radargram.png', dpi=300)